# Classification Model - Random Forests

In [14]:
#Importing all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, make_scorer
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.ensemble import RandomForestClassifier

In [15]:
print(os.getcwd())

c:\Users\ntama\Documentos\Data Science\Codecademy\projects\Date-A-Scientist - Final Project\OKCupid-Date-A-Scientist-Starter\date-a-scientist-project\notebooks


## Importing predictors and target - train, test and validation subsets

In [16]:
X_train = pd.read_csv("../data/X_train.csv", index_col=False)
X_train = X_train.loc[:, ~X_train.columns.str.contains('^Unnamed')]

X_val = pd.read_csv("../data/X_val.csv", index_col=False)
X_val = X_val.loc[:, ~X_val.columns.str.contains('^Unnamed')]

X_test = pd.read_csv("../data/X_test.csv", index_col=False)
X_test = X_test.loc[:, ~X_test.columns.str.contains('^Unnamed')]

y_train = pd.read_csv("../data/y_train.csv", index_col=False)
y_train = y_train.loc[:, ~y_train.columns.str.contains('^Unnamed')]

y_val = pd.read_csv("../data/y_val.csv", index_col=False)
y_val = y_val.loc[:, ~y_val.columns.str.contains('^Unnamed')]

y_test = pd.read_csv("../data/y_test.csv", index_col=False)
y_test = y_test.loc[:, ~y_test.columns.str.contains('^Unnamed')]

C:\Users\ntama\AppData\Local\Temp\ipykernel_16940\4027156877.py:7: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv("../data/X_test.csv", index_col=False)


In [17]:
X_train = X_train.drop(['sex'], axis=1)
X_val = X_val.drop(['sex'], axis=1)
X_test = X_test.drop(['sex'], axis=1)

## Checking class imbalance

In [18]:
print(y_train.value_counts())

body_type
0            19354
1            16398
2             6210
Name: count, dtype: int64


Classes 2 and 3 are minoritarian.

In [19]:
rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_jobs=-1)

# 2. Grid de hiperparámetros (simple para testear rápido)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# 3. Búsqueda con GridSearchCV
grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=make_scorer(f1_score, average='macro'),
    cv=3,
    verbose=2,
    n_jobs=-1
)

# 4. Entrenamiento
grid.fit(X_train, y_train)

# 5. Evaluación en validación
print("Mejores hiperparámetros:")
print(grid.best_params_)

y_val_pred = grid.predict(X_val)
print("Evaluación en validación:")
print(classification_report(y_val, y_val_pred))
print(confusion_matrix(y_val, y_val_pred))

Fitting 3 folds for each of 48 candidates, totalling 144 fits


c:\Users\ntama\anaconda3\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Mejores hiperparámetros:
{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Evaluación en validación:
              precision    recall  f1-score   support

           0       0.56      0.54      0.55      4147
           1       0.43      0.20      0.28      3514
           2       0.21      0.52      0.30      1331

    accuracy                           0.41      8992
   macro avg       0.40      0.42      0.37      8992
weighted avg       0.46      0.41      0.41      8992

[[2229  694 1224]
 [1361  716 1437]
 [ 395  238  698]]
